In [ ]:
import pandas

df = pandas.read_csv(
    "./questions.csv",
    header=0,
    names=[
        "show",
        "air_date",
        "round",
        "category",
        "value",
        "question",
        "answer",
    ],
    converters={
        "value": lambda value: (
            None if value == "None" else float(value.replace("$", "").replace(",", ""))
        ),
        "answer": lambda value: str(value),
    },
    parse_dates=["air_date"],
)

In [93]:
df.sort_values(
    by=["show", "round", "category", "value"],
    ascending=[True, False, True, True],
    inplace=True,
    ignore_index=True,
)
df

,show,air_date,round,category,value,question,answer
0,1,1984-09-10,Jeopardy!,ACTORS & ROLES,100.0,Video in which Michael Jackson plays a werewol...,"""Thriller"""
1,1,1984-09-10,Jeopardy!,ACTORS & ROLES,200.0,"2 ""Saturday Night"" alumni who tried ""Trading P...",Dan Aykroyd & Eddie Murphy
2,1,1984-09-10,Jeopardy!,ACTORS & ROLES,300.0,"He may ""Never Say Never Again"" when asked to b...",Sean Connery
3,1,1984-09-10,Jeopardy!,ACTORS & ROLES,400.0,"The blonde preferred in the film ""Gentlemen Pr...",Marilyn Monroe
4,1,1984-09-10,Jeopardy!,ACTORS & ROLES,500.0,"Sam Shepard played this barrier breaker in ""Th...",Colonel Chuck Yeager
...,...,...,...,...,...,...,...
216925,6300,2012-01-27,Jeopardy!,VISITING THE CITY,200.0,"Experience Little Havana along Calle Ocho, the...",Miami
216926,6300,2012-01-27,Jeopardy!,VISITING THE CITY,400.0,"First the Royal Ontario Museum, then for lunch...",Toronto
216927,6300,2012-01-27,Jeopardy!,VISITING THE CITY,600.0,"Take the ""Sound of Music"" tour or visit the Ha...",Salzburg
216928,6300,2012-01-27,Jeopardy!,VISITING THE CITY,800.0,There's a great opera house on Bennelong Point...,Sydney


In [94]:
df.loc[df["show"] < 3966, "value"] = df.loc[df["show"] < 3966, "value"] * 2
df

,show,air_date,round,category,value,question,answer
0,1,1984-09-10,Jeopardy!,ACTORS & ROLES,200.0,Video in which Michael Jackson plays a werewol...,"""Thriller"""
1,1,1984-09-10,Jeopardy!,ACTORS & ROLES,400.0,"2 ""Saturday Night"" alumni who tried ""Trading P...",Dan Aykroyd & Eddie Murphy
2,1,1984-09-10,Jeopardy!,ACTORS & ROLES,600.0,"He may ""Never Say Never Again"" when asked to b...",Sean Connery
3,1,1984-09-10,Jeopardy!,ACTORS & ROLES,800.0,"The blonde preferred in the film ""Gentlemen Pr...",Marilyn Monroe
4,1,1984-09-10,Jeopardy!,ACTORS & ROLES,1000.0,"Sam Shepard played this barrier breaker in ""Th...",Colonel Chuck Yeager
...,...,...,...,...,...,...,...
216925,6300,2012-01-27,Jeopardy!,VISITING THE CITY,200.0,"Experience Little Havana along Calle Ocho, the...",Miami
216926,6300,2012-01-27,Jeopardy!,VISITING THE CITY,400.0,"First the Royal Ontario Museum, then for lunch...",Toronto
216927,6300,2012-01-27,Jeopardy!,VISITING THE CITY,600.0,"Take the ""Sound of Music"" tour or visit the Ha...",Salzburg
216928,6300,2012-01-27,Jeopardy!,VISITING THE CITY,800.0,There's a great opera house on Bennelong Point...,Sydney


In [40]:
VALUES = {
    "Jeopardy!": [200, 400, 600, 800, 1000],
    "Double Jeopardy!": [400, 800, 1200, 1600, 2000],
}

In [35]:
categories.iloc[0]

Missing    Excess
show round            category                               
1    Double Jeopardy! 4-LETTER WORDS         [1600]  [2000.0]
     Jeopardy!        LAKES & RIVERS          [600]  [1600.0]
2    Double Jeopardy! OPERA                  [1200]  [2000.0]
3    Double Jeopardy! NEW TESTAMENT          [2000]  [3000.0]
                      WORLD OF FOOD          [1200]  [2000.0]
...                                             ...       ...
6297 Jeopardy!        YOU'LL NEED INSURANCE   [800]  [3400.0]
6298 Double Jeopardy! MORE 4 YEARS           [1600]  [2200.0]
                      PLANT SCIENCE           [800]  [6000.0]
     Jeopardy!        BIRD BOOKS              [600]  [2400.0]
6300 Jeopardy!        PANTS                   [800]  [1400.0]

[9474 rows x 2 columns]

In [ ]:
def compare_values(group):
    round_name = group.iloc[0]["round"]
    if round_name not in VALUES:
        return group
    expected = Counter(VALUES[round_name])
    actual = Counter(group["value"])
    missing = (expected - actual).elements()
    excess = (actual - expected).elements()
    for found, supposed in zip(excess, missing):
        group["value"].replace(found, supposed, inplace=True)
    return group


categories = df.groupby(["show", "round", "category"]).apply(compare_values)

In [144]:
categories.reset_index(drop=True, inplace=True)

In [219]:
import sys

i = 0


def remove_duplicates(group):
    global i
    sys.stdout.write("\r%d" % (i / 47200 * 100000))
    sys.stdout.flush()
    i += 1

    dupe_indexes = group.duplicated(subset=["value"], keep="last")
    if not dupe_indexes.sum():
        return group
    round_name = group.iloc[0]["round"]
    replacement = next(
        (value for value in VALUES[round_name] if value not in group["value"].values),
        None,
    )
    group["value"][dupe_indexes] = [replacement]
    return group


groups = df.groupby(["show", "round", "category"]).apply(remove_duplicates)

99997

show   air_date  \
show round            category                                    
1    Double Jeopardy! '50'S TV          24         1 1984-09-10   
                                        25         1 1984-09-10   
                                        26         1 1984-09-10   
                                        27         1 1984-09-10   
                                        28         1 1984-09-10   
...                                              ...        ...   
6300 Jeopardy!        VISITING THE CITY 216925  6300 2012-01-27   
                                        216926  6300 2012-01-27   
                                        216927  6300 2012-01-27   
                                        216928  6300 2012-01-27   
                                        216929  6300 2012-01-27   

                                                           round  \
show round            category                                     
1    Double Jeopardy! '50'S TV          24      Double Jeopardy!   
                                        25      Double Jeopardy!   
                                        26      Double Jeopardy!   
                                        27      Double Jeopardy!   
                                        28      Double Jeopardy!   
...                                                          ...   
6300 Jeopardy!        VISITING THE CITY 216925         Jeopardy!   
                                        216926         Jeopardy!   
                                        216927         Jeopardy!   
                                        216928         Jeopardy!   
                                        216929         Jeopardy!   

                                                         category   value  \
show round            category                                              
1    Double Jeopardy! '50'S TV          24               '50'S TV   400.0   
                                        25               '50'S TV   800.0   
                                        26               '50'S TV  1200.0   
                                        27               '50'S TV  1600.0   
                                        28               '50'S TV  2000.0   
...                                                           ...     ...   
6300 Jeopardy!        VISITING THE CITY 216925  VISITING THE CITY   200.0   
                                        216926  VISITING THE CITY   400.0   
                                        216927  VISITING THE CITY   600.0   
                                        216928  VISITING THE CITY   800.0   
                                        216929  VISITING THE CITY  1000.0   

                                                                                         question  \
show round            category                                                                      
1    Double Jeopardy! '50'S TV          24      Occupation of Richard Diamond, Peter Gunn & Mi...   
                                        25                              She was "Our Miss Brooks"   
                                        26      Amount Michael Anthony gave out each week on b...   
                                        27                  His card read "Have gun, will travel"   
                                        28      Name under which experimenter Don Herbert taug...   
...                                                                                           ...   
6300 Jeopardy!        VISITING THE CITY 216925  Experience Little Havana along Calle Ocho, the...   
                                        216926  First the Royal Ontario Museum, then for lunch...   
                                        216927  Take the "Sound of Music" tour or visit the Ha...   
                                        216928  There's a great opera house on Bennelong Point...   
                                        216929  Arriving at the train station

In [238]:
df[29:34]

,show,air_date,round,category,value,question,answer
29,1,1984-09-10,Jeopardy!,ACTORS & ROLES,1000.0,"Sam Shepard played this barrier breaker in ""Th...",Colonel Chuck Yeager
30,1,1984-09-10,Jeopardy!,ANIMALS,200.0,These rodents first got to America by stowing ...,rats
31,1,1984-09-10,Jeopardy!,ANIMALS,400.0,"There are about 40,000 muscles & tendons in th...",the trunk
32,1,1984-09-10,Jeopardy!,ANIMALS,600.0,"When husbands ""pop"" for an ermine coat, they'r...",weasel
33,1,1984-09-10,Jeopardy!,ANIMALS,800.0,"Close relative of the pig, though its name mea...",the hippopotamus


In [234]:
groups.reset_index(drop=True, inplace=True)

In [240]:
df = groups

In [241]:
df[df.duplicated(["show", "round", "category", "value"])]

,show,air_date,round,category,value,question,answer


In [243]:
df.to_csv("./questions_cleaned.csv", index=False)